### Importing packages

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning

###  Load the pretrained Network

In [3]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

###  Freeze the layers

In [12]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

###  Training parameters

In [13]:
img_width, img_height = 150, 150
train_data_dir = 'train'
val_data_dir = 'validation'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 5
nb_val_samples = 6
nb_epochs = 5

###  Build a classification model on top of Base Network


In [14]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(input=input, output=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

###  Dataset Preparation

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

Found 5 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


###  Training

In [17]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, samples_per_epoch=nb_train_samples, 
                              nb_epoch=nb_epochs, validation_data=validation_generator, nb_val_samples=nb_val_samples)

print('Training Completed!')

Epoch 1/5
1/1 [==============================] - 46s - loss: 3.3801 - acc: 0.0000e+00 - val_loss: 1.4159 - val_acc: 0.5000
Epoch 2/5
1/1 [==============================] - 51s - loss: 0.8242 - acc: 0.7500 - val_loss: 0.4698 - val_acc: 0.6667
Epoch 3/5
1/1 [==============================] - 30s - loss: 2.1100e-05 - acc: 1.0000 - val_loss: 0.4557 - val_acc: 0.6667
Epoch 4/5
1/1 [==============================] - 28s - loss: 0.0459 - acc: 1.0000 - val_loss: 0.5350 - val_acc: 0.6667
Epoch 5/5
1/1 [==============================] - 24s - loss: 3.5167e-05 - acc: 1.0000 - val_loss: 0.6649 - val_acc: 0.6667
Training Completed!


###  Test the model

In [18]:
img_path = 'dog.jpg'
label = ['Cat','Dog']
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features == 1)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

Predicted Array: [[0. 1.]]
Predicted Label: Dog


### Models for image classification with weights trained on ImageNet
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* DenseNet
* NASNet

In [19]:
model.save('catdog.h5')

In [20]:
pwd

'/home/nbuser/library/Image_processing'